# Лабораторная работа по Python

В этой тетради мы познакомимся с простым планировщиком замены оборудования и одновременно потренируемся писать функции на Python. Все необходимые функции приведены ниже, поэтому тетрадь можно запускать отдельно от репозитория.

## Подготовка
Импортируем нужные модули и опишем класс `Config` с параметрами модели.

In [ ]:
from dataclasses import dataclass, field
from typing import List, Tuple
import numpy as np

In [ ]:
@dataclass
class Config:
    num_objects: int = 1000
    regional_centers: List[Tuple[float, float]] = field(default_factory=lambda: [(0, 0), (100, 0), (0, 100), (100, 100)])
    network_type: str = 'star'  # 'star' или 'spider'
    warehouses: int = 1
    working_hours: int = 10
    replace_minutes: int = 70
    car_capacity: int = 16
    speed_kmh: int = 50
    car_cost_per_day: float = 8000
    engineer_salary: float = 80000
    driver_salary: float = 65000
    hotel_cost: float = 2000
    allowance_cost: float = 1000

## Расстояние между точками
Ниже нужно реализовать функцию `distance`. Используйте теорему Пифагора.

In [ ]:
def distance(a, b):
    """Евклидово расстояние между двумя точками."""
    # TODO: верните корень квадратный из суммы квадратов разностей координат
    return ((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2) ** 0.5  # ответ

## Матрица расстояний
Эту функцию часто просят написать на собеседовании. Попробуйте сделать это самостоятельно, затем сравните с ответом в комментарии.

In [ ]:
def distance_matrix(points):
    """Матрица попарных расстояний."""
    # TODO: заполните двумерный массив расстояний
    n = len(points)
    mat = np.zeros((n, n))
    for i in range(n):
        for j in range(i + 1, n):
            d = distance(points[i], points[j])
            mat[i, j] = mat[j, i] = d
    return mat
    # ответ: цикл по i,j и вычисление distance

## Генерация и назначение объектов
Теперь подготовим функции, которые создают объекты и распределяют их по региональным центрам.

In [ ]:
def generate_objects(cfg: Config):
    rng = np.random.default_rng(0)
    centers = np.array(cfg.regional_centers)
    objects = []
    for _ in range(cfg.num_objects):
        idx = rng.integers(0, len(centers))
        center = centers[idx]
        offset = rng.normal(scale=20, size=2)
        objects.append(center + offset)
    return np.array(objects)

In [ ]:
def assign_objects_to_centers(objects, cfg: Config):
    centers = np.array(cfg.regional_centers)
    assignments = []
    for obj in objects:
        dists = np.linalg.norm(centers - obj, axis=1)
        assignments.append(np.argmin(dists))
    return np.array(assignments)

## Оценка сценария
Осталось несколько простых функций для подсчёта количества бригад и стоимости.

In [ ]:
def objects_per_day(cfg: Config, avg_distance_km: float):
    travel_minutes = (avg_distance_km * 2 / cfg.speed_kmh) * 60
    total_minutes = cfg.replace_minutes + travel_minutes
    max_objects = (cfg.working_hours * 60) // total_minutes
    return int(min(cfg.car_capacity, max_objects))

In [ ]:
def required_crews(cfg: Config, months: int, avg_distance_km: float):
    objs_day = objects_per_day(cfg, avg_distance_km)
    if objs_day == 0:
        return float('inf')
    capacity = objs_day * months * 30
    crews = int(np.ceil(cfg.num_objects / capacity))
    return max(crews, 1)

In [ ]:
def cost_estimate(cfg: Config, months: int, avg_distance_km: float):
    crews = required_crews(cfg, months, avg_distance_km)
    days = months * 30
    wages = crews * (cfg.engineer_salary + cfg.driver_salary) * (months / 1)
    cars = crews * cfg.car_cost_per_day * days
    hotels = crews * cfg.hotel_cost * days
    allowance = crews * cfg.allowance_cost * days
    return {
        'бригады': crews,
        'зарплата': wages,
        'автомобили': cars,
        'гостиницы': hotels,
        'командировочные': allowance,
        'итого': wages + cars + hotels + allowance,
    }

In [ ]:
def average_distance(objects, assignments, cfg: Config):
    centers = np.array(cfg.regional_centers)
    factor = 1.3 if cfg.network_type == 'star' else 1.0
    if cfg.warehouses == 1:
        wh = centers[0]
        dists = np.linalg.norm(objects - wh, axis=1) * factor
        return float(np.mean(dists))
    total = 0.0
    for center_idx in range(len(centers)):
        idxs = np.where(assignments == center_idx)[0]
        if len(idxs) == 0:
            continue
        dist = np.linalg.norm(objects[idxs] - centers[center_idx], axis=1) * factor
        total += np.sum(dist)
    return total / len(objects)

In [ ]:
def solve_scenario(cfg: Config, months: int):
    objects = generate_objects(cfg)
    assignments = assign_objects_to_centers(objects, cfg)
    avg_dist = average_distance(objects, assignments, cfg)
    result = cost_estimate(cfg, months, avg_dist)
    return result

In [ ]:
def run_variants(months_options):
    rows = []
    for net in ['star', 'spider']:
        for wh in [1, 4]:
            cfg = Config(network_type=net, warehouses=wh)
            for m in months_options:
                res = solve_scenario(cfg, m)
                rows.append({'network_type': net, 'warehouses': wh, 'months': m, **res})
    return rows

## Пример расчёта
Создадим конфигурацию по умолчанию и посмотрим на результаты работы функций.

In [ ]:
cfg = Config()
objects = generate_objects(cfg)
print('Всего объектов:', len(objects))
assignments = assign_objects_to_centers(objects, cfg)
res = solve_scenario(cfg, months=2)
print('Итог для срока 2 месяца:', res['итого'])

## Задания для самостоятельной работы
1. Измените значения в `Config` (например, `speed_kmh` или `engineer_salary`) и пересчитайте стоимость.
2. Добавьте ещё один срок выполнения в функцию `run_variants` и проанализируйте изменения.
3. Измените число складов `warehouses` и сравните результаты.

Ответы и полученные таблицы включите в отчёт.